In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
from datetime import datetime, timedelta
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

gs_challenge = gc.open('NYUAD Swim Challenge 2 (Responses)').sheet1
gs_teams = gc.open('Team Registration for NYUAD Swim Challenge 2 (Responses)').sheet1
gs_challenge1 = gc.open('NYUAD Swim Challenge 1 (Responses)').worksheet("clean_data")


KeyboardInterrupt: 

In [ ]:
#TEAM REGISTRATION
#going to create sample team registrations
#drop nans
df = (pd.DataFrame.from_records(gs_teams.get_all_values())
      .replace('', np.NaN)
      .dropna())

col = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
df = df.rename(columns=col)


#reanming columns
df.columns = [x.strip().lower().replace(" ", "_") for x in df.columns]

for i in [1,2,3]:
  df['netid_-_member_' + str(i)] = df['netid_-_member_' + str(i)].str.lower().str.strip()

df

In [ ]:
#two dictionaries
#dictionary for netid - team
dic_id_team = {}
#dictionary for team - members (will display this as a table in the website, just to show who has registered)
dic_team_names = {}

def registerteam (row):
  #append to dic netid - team
  dic_id_team[row.loc['netid_-_member_1']] = row.loc['team_name']
  dic_id_team[row.loc['netid_-_member_2']] = row.loc['team_name']
  dic_id_team[row.loc['netid_-_member_3']] = row.loc['team_name']
  #append to dic team - names
  dic_team_names[row.loc['team_name']] = [row.loc['first_name_-_member_1'], row.loc['first_name_-_member_2'], row.loc['first_name_-_member_3']]

#create dictionaries
df.apply(registerteam,axis=1)

#print dicitonaries
print(dic_id_team)
print(dic_team_names)

#OUTPUT TABLE - team reigstration
df_teams = pd.DataFrame(dic_team_names).T


print(df_teams.to_markdown())



In [ ]:
#CHALLENGE

#load df
#construct Nans
df = (pd.DataFrame.from_records(gs_challenge.get_all_values())
      .replace('', np.NaN))
col = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
df = df.rename(columns=col)

#reanming columns
df.columns = [x.strip().lower().replace(" ", "_") for x in df.columns]
df.rename(columns = {'leaderboard_nickname':'leaderboard', 'did_you_swim_more_than_once_today?': 'swam_more_than_once'}, inplace = True)

#drop empty rows
df = df.dropna(subset=['timestamp', 'netid', 'first_name', 'last_name', 'distance', 'date_you_swam', 'class'])

#lowercase netid and strip
df.netid = df['netid'].str.lower().str.strip()
#lower case class
df['class'] = df['class'].str.lower()
#replace @nyu edu if someone puts email instead of id
df['netid'] = df['netid'].astype(str).str.lower().replace('@.*', '', regex=True)

#replace leaderboard entires
df['leaderboard'] = df['leaderboard'].replace('Yes.*', '1', regex= True)
df['leaderboard'] = df['leaderboard'].replace('No.*', '0', regex= True)
df['leaderboard'] = pd.to_numeric(df.leaderboard)
#make distance a interger
df.distance = pd.to_numeric(df.distance)

#timestamp conversion - doesnt work in python shell - not sure why
df.timestamp = pd.to_datetime(df.timestamp)
df.date_you_swam = df.date_you_swam.apply(lambda x: x.split('(')[0].strip()+'-2021')
df.date_you_swam = pd.to_datetime(df.date_you_swam , format='%b %d-%Y')
df

In [ ]:
df.dtypes

In [ ]:
for k, sdf in df.groupby(['netid', 'date_you_swam']):
  if len(sdf)>1:
    print(sdf[['timestamp', 'netid', 'first_name', 'distance', 'date_you_swam', 'time']])



In [ ]:
#anonymize
cute_nicknames = ['Doplhin on a Study Away',
                  'Falcon with Fins',
                  'Turbo Gazelle',
                  'Baby Shark',
                  'Brave Turtle',
                  'Little Penguin',
                  'Vegan Shark',
                  'Commited Clownfish',
                  'Nemo looking for their major']

random.seed(0)

def anonym(row):
  if row['leaderboard']  == 0:
    row['first_name'] = random.choice(cute_nicknames) + ' '+ str(ord(row['netid'][0]) - 96)
  else:
    row['first_name'] = row['first_name'] + ' ' +  row['last_name'][0]
  return row

df = df.apply(anonym, axis= 1)
df

In [ ]:
#removing duplicate data
#df_removed_duplicated = df.drop_duplicates(subset=['netid', 'date_you_swam', 'distance'], keep='last')

# df_droplog = pd.DataFrame()
# mask = df.duplicated(subset=['netid', 'date_you_swam', 'distance'], keep='first')
# df_keep = df.loc[~mask]
# df_droplog = df_droplog.append(df.loc[mask])

# df_keep
# df_droplog



In [ ]:
def distance_today(i):

  today = df[df.date_you_swam == ('2021-04-'+ i )].distance.sum()
  return round(today/1000)




In [ ]:
#assign teams
def teamcol(id):
  if id in dic_id_team:
    return dic_id_team[id]
  else:
    return np.NaN
df['team'] = df.netid.apply(teamcol)

#print(df[['team', 'distance']])
#groupby teams
#OUTPUT TABLE - team rankings
rank_teams = df.groupby('team').agg({'distance': 'sum'}).sort_values(by = 'distance',ascending=False)


print(rank_teams.to_markdown())

In [ ]:
rank_teams_byin = df.groupby(['team', 'netid']).agg({'distance': 'sum'}).reset_index(drop=False)

ndf = {}
def mytransform(sdf):
  #print(sdf)
  #print('----')
  ndf[sdf['team'].iloc[0]] = sdf['distance'].T.to_list()
  return sdf

rank_teams_byin.groupby(['team']).apply(mytransform)

ndf = pd.DataFrame.from_dict(ndf, orient='index', columns = ['member 1', 'member 2', 'member 3'])

ndf['total'] = ndf.apply(lambda row: row.sum(), axis = 1)
ndf = ndf.sort_values(by = 'total',ascending=False)
sns.set()
sns.set_style("whitegrid")
sns.set_palette('pastel')


ndf[['member 1', 'member 2', 'member 3']].plot(kind='bar', stacked=True);
labels = ['MVM', 'M & Colombians', 'Macrisgi', 'Bananas', '7AMs', 'Chrząszcze', 'Chonks']
plt.gca().set_xticklabels(labels);
plt.title('Distance Swam by Team in Meters', alpha = 0.9);



In [ ]:
#group by individual
rank_individual = df.groupby('netid').agg({'distance':'sum', 'first_name': 'first'}).sort_values(by=['distance'], ascending=False).reset_index()
#rank_individual['full_name'] = rank_individual.apply(lambda row: row.first_name+' '+ row.last_name[0] , axis = 1)
rank_individual['total_sessions'] = 0
rank_individual['swim_session_dates'] = ''


all_swimmers = df.netid.unique()

def numsessions(row):
  id = row['netid']
  dates_list = df[df.netid == id].date_you_swam.array.strftime("%d").tolist()
  string = ''
  for d in dates_list:
    string = string +  str(d) + '-'
  row['swim_session_dates'] = string
  row['total_sessions'] = len(df[df.netid == row['netid']])
  return row
rank_individual = rank_individual.apply(numsessions, axis = 1)

rank_individual = rank_individual.drop(columns=[ 'netid'])
rank_individual.index = np.arange(1, len(rank_individual) + 1)
#OUTPUT TABLE - rank individuals
print(rank_individual.sort_values(by = 'distance',ascending=False).to_markdown())


In [ ]:
now = datetime.now()+ timedelta(hours=4)
print('# NYUAD Swim Challenge Leaderboard  ')
print('April 5 - 11, 2021  ')
print()
print('Updated: ')
print(now.strftime("%d/%m/%Y %H:%M"))
print()
#print('Distance swam today: ' + str(distance_today(datetime.now().strftime("%d")))+ ' kilometers  ')
print('## Distance swam in total: ' + str(round(df.distance.sum()/1000)) + ' kilometers  ')
print()
#print('Number of swimmers today: ' + str(len(df[df.date_you_swam == ('2021-04-'+ datetime.now().strftime("%d") )])) + '  ')
print('Participants in total: ' + str(len(rank_individual)) + '  ')
print()

print('## Team Ranking  ')
print('if you win team prize, you cannot win individual prize')
print(rank_teams.to_markdown())

print('## Individual Ranking  ')
print(rank_individual.sort_values(by = 'distance',ascending=False).to_markdown())


print('## Teams registered  ')
print(df_teams.to_markdown())

In [ ]:
rank_teams


In [ ]:

dic_dist_day = {}
for i in [5,6,7,8,9,10,11]:
  i_s = str(i)
  dic_dist_day['2021-04-'+ i_s] = distance_today(i_s)
dic_dist_day

distance_daily = pd.DataFrame.from_dict(dic_dist_day, orient='index', columns = ['Distance swam in kilometers'])
distance_daily.index=pd.to_datetime(distance_daily.index)
distance_daily.sort_index().cumsum().plot()
ax = plt.gca()
plt.legend(["Cumulative Distance Swam in Kilometers"]);

# #good name and soften
plt.title('NYUAD Swim Challenge 2', alpha = 0.9)
# plt.xticks(pos, languages, alpha=0.8)

# #removing ticks
#plt.tick_params(bottom='off', left= 'off', labelleft = 'off', labelbottom = 'off')

#remove frames
for spine in plt.gca().spines.values():
  spine.set_color(c = 'lightgray')



In [ ]:
#load df

#construct Nans
df = (pd.DataFrame.from_records(gs_challenge1.get_all_values())
      .replace('', np.NaN))
col = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
df = df.rename(columns=col)

#reanming columns
df.columns = [x.strip().lower().replace(" ", "_") for x in df.columns]
df.rename(columns = {'leaderboard_nickname':'leaderboard', 'did_you_swim_more_than_once_today?': 'swam_more_than_once'}, inplace = True)

#drop empty rows
df = df.dropna(subset=['timestamp', 'netid', 'first_name', 'last_name', 'distance', 'date_you_swam', 'class'])

#lowercase netid and strip
df.netid = df['netid'].str.lower().str.strip()
#lower case class
df['class'] = df['class'].str.lower()
#replace @nyu edu if someone puts email instead of id
df['netid'] = df['netid'].astype(str).str.lower().replace('@.*', '', regex=True)
#cast distance to numeric
df['distance'] = df['distance'].astype(str).str.lower().replace(',', '', regex=True).astype(int)


#group by individual
individual_rank_challenge_1 = df.groupby('netid').agg({'distance':'sum', 'first_name': 'first', 'last_name': 'first'}).sort_values(by=['distance'], ascending=False).reset_index()
individual_rank_challenge_1.rename(columns = {'distance': 'distance1'}, inplace = True)

individual_rank_challenge_1








In [ ]:
#CHALLENGE2 ranking for most imporved

#load df
#construct Nans
df = (pd.DataFrame.from_records(gs_challenge.get_all_values())
      .replace('', np.NaN))
col = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
df = df.rename(columns=col)

#reanming columns
df.columns = [x.strip().lower().replace(" ", "_") for x in df.columns]
df.rename(columns = {'leaderboard_nickname':'leaderboard', 'did_you_swim_more_than_once_today?': 'swam_more_than_once'}, inplace = True)

#drop empty rows
df = df.dropna(subset=['timestamp', 'netid', 'first_name', 'last_name', 'distance', 'date_you_swam', 'class'])

#lowercase netid and strip
df.netid = df['netid'].str.lower().str.strip()
#lower case class
df['class'] = df['class'].str.lower()
#replace @nyu edu if someone puts email instead of id
df['netid'] = df['netid'].astype(str).str.lower().replace('@.*', '', regex=True)

#replace leaderboard entires
df['leaderboard'] = df['leaderboard'].replace('Yes.*', '1', regex= True)
df['leaderboard'] = df['leaderboard'].replace('No.*', '0', regex= True)
df['leaderboard'] = pd.to_numeric(df.leaderboard)
#make distance a interger
df.distance = pd.to_numeric(df.distance)

#timestamp conversion - doesnt work in python shell - not sure why
df.timestamp = pd.to_datetime(df.timestamp)
df.date_you_swam = df.date_you_swam.apply(lambda x: x.split('(')[0].strip()+'-2021')
df.date_you_swam = pd.to_datetime(df.date_you_swam , format='%b %d-%Y')

#group by individual
individual_rank_challenge_2 = df.groupby('netid').agg({'distance':'sum', 'first_name': 'first', 'last_name': 'first'}).sort_values(by=['distance'], ascending=False).reset_index()
individual_rank_challenge_2.rename(columns = {'distance': 'distance2'}, inplace = True)

individual_rank_challenge_2

In [ ]:

most_improved = individual_rank_challenge_1.merge(individual_rank_challenge_2, how='inner', on='netid')
most_improved['improvement'] = (most_improved.distance2 - most_improved.distance1)/most_improved.distance1
most_improved.sort_values(by='improvement', ascending=False)
